<a href="https://colab.research.google.com/github/HuaChan233/NovelAI/blob/main/NovelAILeaks_API_Backend_(4chan_Ver_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

基于 4chan 魔改版 NovelAILeaks (naifu) 制作。[来源](https://boards.4channel.org/g/thread/89095460#p89097704)

使用官方前端 + 优化版后端，可突破75限制，支持所有模型。

Credit: https://t.me/StableDiffusion_CN https://t.me/exlolicon

Thanks: Anonymous, 炼铜术士, 神楽坂早苗️, Jonathan, 咕 咕, 猫又逆变器, Gaein nidb



0. 检查 GPU 工作状态

In [1]:
!nvidia-smi

Tue Oct 11 13:11:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

1. 下载 Novel AI API 后端、模型 （如果下载速度太慢可尝试 restart）

In [ ]:
%cd /content/
!apt install -y -qq aria2
!aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z \
  https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar \
  https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.tar 
  
!tar xf naifu.tar && rm naifu.tar

2. 安装依赖

In [ ]:
%cd /content/naifu
!pip install virtualenv && bash ./setup.sh
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin
!curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared

3. 启动模型，访问输出的映射地址（以 `trycloudflare.com` / `bore.pub` 结尾）即可。
   - 请等待模型加载完成（出现`Application startup complete`字样）再访问
   - cloudflare 提供的服务偶尔会出现请求超时，可换用 bore 隧道

In [ ]:
%cd /content/naifu
!sed -i 's/# export SAVE_FILES="1"/export SAVE_FILES="1"/g' run.sh
!bash run.sh & cloudflared tunnel --url localhost:6969 

4. (可选）默认使用的是 4G 大小的 `animefull-final-pruned` 模型。如果想使用 7G 的 `animefull-latest` 模型，运行这个

In [ ]:
%cd /content/
!tar xf animefull-latest.tar -C /content/naifu/models && rm animefull-latest.tar
!sed -i 's/map_location="cpu"/map_location="cuda"/g' /content/naifu/hydra_node/models.py

%cd /content/naifu
%env DTYPE=float16
%env CLIP_CONTEXTS=3
%env AMP=1
%env MODEL=stable-diffusion
%env DEV=True
%env MODEL_PATH=models/animefull-latest
%env ENABLE_EMA=1
%env VAE_PATH=models/animevae.pt
%env PENULTIMATE=1
%env PYTHONDONTWRITEBYTECODE=1
%env SAVE_FILES=1

!./venv/bin/python -m uvicorn --host 0.0.0.0 --port=6969 main:app & bore local 6969 --to bore.pub & cloudflared tunnel --url localhost:6969 